Selenium para descargar base de dato de la intranet de Opción de Energía

## Librerías

In [5]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys

## Page 1 - Login Opcion Comercializadora Intranet

In [6]:
driver = webdriver.Chrome("../Programas/chromedriver")

In [7]:
url = "https://canalagentes.opcionenergia.com/authentication/login"

In [8]:
driver.get(url)

In [9]:
driver.find_element_by_name("user").send_keys("vivolt")
driver.find_element_by_name("password").send_keys("temporal01vivolt")
driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)

## Page 2 - Download "SIPS"

Para asignar la variable busco por el xpath, y desde tools de chrome, abro el codigo html y copio el xpath. 

Pulsando el boton de SIPS:

In [10]:
sips = driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li')

In [11]:
sips.click()

rellenando el CUPS: # agregar funcion para rellenar el cups desde la API

In [12]:
driver.find_element_by_name("Cups").send_keys("ES0021000005204736CV")

In [14]:
mostrar_grafico_sips = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button')

In [15]:
mostrar_grafico_sips.click()

In [16]:
descargar_csv = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button')

In [17]:
descargar_csv.click()

## Funcion para el proceso

In [27]:
from config import user_data

ModuleNotFoundError: No module named 'config'

In [ ]:
class Download_Consumo_Cliente:
    # class variable (vs instance variable)?¿?¿
    url = "https://canalagentes.opcionenergia.com/authentication/login"
    
    def __init__(self, chromedriver, user_data):
        self.driver = chromedriver
        self.user_data = user_data
    
    def access_form(self):
        self.driver.get(self.url)
        seg_social_square = driver.find_element_by_css_selector("div div div input")
        seg_social_square.click()